In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import prepare as prep
import acquire as acq
import explore as exp


import env 

In [2]:
df = acq.get_traffic_data()

In [3]:
df = prep.prep_traffic(df)

/Users/esayasa./Desktop/Git_repositories/individual-project-1/prepare.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=column_mapping, inplace=True)
/Users/esayasa./Desktop/Git_repositories/individual-project-1/prepare.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['City'], inplace=True)
/Users/esayasa./Desktop/Git_repositories/individual-project-1/prepare.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [ ]:
df.rename(columns=column_mapping, inplace=True)
df.dropna(subset=['City'], inplace=True)
df.dropna(subset=['Zipcode'], inplace=True)
df.rename(columns=lambda x: x.lower(), inplace=True)
df.columns = df.columns.str.lower()
df.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng', 'timezone', 'airport_code', 'weather_timestamp', 
               'source', 'civil_twilight', 'nautical_twilight', 'astronomical_twilight', 'description'], inplace=True)
df.replace({True: 1, False: 0}, inplace=True)
desired_date = '2021-01-01'
df['start_time'] >= desired_date
df['start_time'] = pd.to_datetime(df['start_time'], format='%Y-%m-%d')
df['end_time'] = pd.to_datetime(df['end_time'], format='%Y-%m-%d')
df['start_date'] = df['start_time'].dt.date
df['start_date'] = pd.to_datetime(df['start_date'], format='%Y-%m-%d')
df['start_time_'] = df['start_time'].dt.time
df['end_date'] = df['end_time'].dt.date
df['end_date'] = pd.to_datetime(df['end_date'], format='%Y-%m-%d')
df['day_of_week'] = df['start_time'].dt.weekday
df['month'] = df['start_time'].dt.month
df['len_of_affect'] = df.end_time - df.start_time
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Unknown'
df['season'] = df['start_date'].dt.month.apply(get_season)
df= df.sort_values(by=['zipcode', 'street'], ascending=[False, True])
#df.drop(columns=['end_time', 'end_date', 'start_time', 'start_date'], axis=1, inplace=True)
df.replace({'Day': 1, 'Night': 0}, inplace=True)


In [4]:
df.shape

(2323662, 40)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2323662 entries, 4426557 to 5424907
Data columns (total 40 columns):
 #   Column             Dtype          
---  ------             -----          
 0   severity           int64          
 1   start_time         datetime64[ns] 
 2   end_time           datetime64[ns] 
 3   distance           float64        
 4   street             object         
 5   city               object         
 6   county             object         
 7   state              object         
 8   zipcode            object         
 9   country            object         
 10  temp               float64        
 11  wind_chill         float64        
 12  humidity           float64        
 13  pressure           float64        
 14  visibility         float64        
 15  wind_direction     object         
 16  wind_speed_mph     float64        
 17  precipitation      float64        
 18  weather_condition  object         
 19  amenity            int64          
 

In [6]:
df = prep.remove_outliers(df)

#got rid of outliers across all continuous variable features of impact outside of 3 standard deviations
columns_to_filter = ['distance', 'temp', 'wind_chill', 'humidity', 'pressure', 'visibility', 'wind_speed_mph', 'precipitation', 'len_of_affect']

# Set the threshold for outlier detection (e.g., z-score > 3 is considered an outlier)
threshold = 3

# Calculate z-scores for the specified columns
z_scores = df[columns_to_filter].apply(lambda x: np.abs((x - x.mean()) / x.std()))

# Filter the DataFrame to exclude rows with outliers in any of the columns
df = df[(z_scores <= threshold).all(axis=1)]


#getting rid of weather conditions that don't happen enough to be of any significant impact on the dataset since there are 144 weather condition classificaitons

condition_counts = df['weather_condition'].value_counts()

# Step 2: Filter out the weather conditions that occur less than 70,000 times
filtered_conditions = condition_counts[condition_counts >= 70000].index

# Step 3: Remove the rows corresponding to the filtered weather conditions
df_filtered = df[df['weather_condition'].isin(filtered_conditions)]

In [7]:
df.shape

(1329374, 40)

desired_date = pd.to_datetime('2021-01-01')

# Filter the DataFrame to include rows from the desired date onward
df_filtered = df[df['start_time'] >= desired_date]


In [8]:
df.start_time.max()

Timestamp('2023-03-31 23:25:30')

In [9]:
df.start_time.min()

Timestamp('2016-03-23 05:03:47')

In [10]:
df.distance.min()

0.0

In [11]:
df.len_of_affect.mean()
#what is distance?

Timedelta('0 days 01:51:29.718939139')

In [12]:
df.head(1)


,severity,start_time,end_time,distance,street,city,county,state,zipcode,country,...,traffic_signal,turning_loop,sunrise_sunset,start_date,start_time_,end_date,day_of_week,month,len_of_affect,season
2464929,2,2018-11-22 19:51:38,2018-11-22 20:20:46,0.0,N Shore Rd,Truckee,Nevada,CA,96161-0348,US,...,1,0,0.0,2018-11-22,19:51:38,2018-11-22,3,11,0 days 00:29:08,Autumn


# AC (After Conversion (of dataframe to csv))

In [13]:
#the originial dataset was too large to keep incidentally starting over on/running all kernels so I created a csv of the data after cutting out over 5 million rows


file_path = 'output.csv'
df.to_csv(file_path, index=False)

In [14]:
df = pd.read_csv('output.csv')

get rid of outliers for distance, and len_of_affect
bin all continuous value columns, index identification
get values for weather condition

In [15]:
df.head(1)

,severity,start_time,end_time,distance,street,city,county,state,zipcode,country,...,traffic_signal,turning_loop,sunrise_sunset,start_date,start_time_,end_date,day_of_week,month,len_of_affect,season
0,2,2018-11-22 19:51:38,2018-11-22 20:20:46,0.0,N Shore Rd,Truckee,Nevada,CA,96161-0348,US,...,1,0,0.0,2018-11-22,19:51:38,2018-11-22,3,11,0 days 00:29:08,Autumn


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329374 entries, 0 to 1329373
Data columns (total 40 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   severity           1329374 non-null  int64  
 1   start_time         1329374 non-null  object 
 2   end_time           1329374 non-null  object 
 3   distance           1329374 non-null  float64
 4   street             1326765 non-null  object 
 5   city               1329374 non-null  object 
 6   county             1329374 non-null  object 
 7   state              1329374 non-null  object 
 8   zipcode            1329374 non-null  object 
 9   country            1329374 non-null  object 
 10  temp               1329374 non-null  float64
 11  wind_chill         1329374 non-null  float64
 12  humidity           1329374 non-null  float64
 13  pressure           1329374 non-null  float64
 14  visibility         1329374 non-null  float64
 15  wind_direction     1329366 non-n

In [17]:
df.shape


(1329374, 40)

must do calculation to create accident index

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329374 entries, 0 to 1329373
Data columns (total 40 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   severity           1329374 non-null  int64  
 1   start_time         1329374 non-null  object 
 2   end_time           1329374 non-null  object 
 3   distance           1329374 non-null  float64
 4   street             1326765 non-null  object 
 5   city               1329374 non-null  object 
 6   county             1329374 non-null  object 
 7   state              1329374 non-null  object 
 8   zipcode            1329374 non-null  object 
 9   country            1329374 non-null  object 
 10  temp               1329374 non-null  float64
 11  wind_chill         1329374 non-null  float64
 12  humidity           1329374 non-null  float64
 13  pressure           1329374 non-null  float64
 14  visibility         1329374 non-null  float64
 15  wind_direction     1329366 non-n

In [ ]:
from geopy.geocoders import Nominatim
